<a href="https://colab.research.google.com/github/takatakamanbou/Data/blob/2023/ex15notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data2023 ex15notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/Data-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?Data/2023



## 準備



以下，コードセルを上から順に実行していってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# 正規分布の確率密度関数
def Gaussian(x, mu, sigma):
    return np.exp(-(x-mu)*(x-mu)/(2*sigma*sigma)) / np.sqrt(2*np.pi*sigma*sigma)

In [ ]:
### サイコロを N 回ふって出た目の数の配列を返す関数
# デフォルトでは，どの目も等しい確率で出る
#
def dice(N, P=np.ones(6)/6.0):

    assert np.isclose(np.sum(P), 1.0)

    X = np.zeros(N, dtype=int)
    P2 = np.cumsum(P)
    for n in range(N):
        a = np.random.random() # [0, 1) の一様乱数
        X[n] = 6
        for i in range(6):
            if a < P2[i]:
                X[n] = i+1
                break

    return X

----
## 「データ分析」から「確率統計I,II,III」へ
----

2年前期の「確率統計I」や3年後期「確率統計II/III」では，データサイエンスの理論的な基盤である「確率学」・「統計学」について学びます．

----
### 記述統計 vs 統計的推測・推計統計

「データ分析」で扱った分析の手法は，

- 入手したデータの分布の様子を調べる（平均，分散，ヒストグラム，etc.）
- 入手したデータの2つの変数の間の関係を調べる（相関係数，散布図，回帰分析，etc）

というものでした．いずれも，「データそのものの性質」を調べている，と言ってもよいでしょう．
第1回の授業でも説明しましたが，このような統計の分野・方法を「記述統計」と言います（注）．

<font size="-1">注） 本当は回帰分析は推計統計に入るのですが，この授業ではそこまで踏み込んだ話はしませんでした．</font>

一方で，データを深く分析しようとすると，「データの裏に隠れた仕組み」を考えてあれこれしたくなってきます．
ものすごくいい加減な言い方ですが，そのようなことを考える統計の分野・方法を「推計統計」あるいは「統計的推測」と言います．

データの裏に **確率分布** を仮定して，データ（標本）はその確率分布にしたがって生み出されたものと考える...といったことをします．高校数学で学んだ **正規分布** などは，確率分布の代表例です．また，高校数学で学んだ **統計的検定** は，まさしく統計的推測の手法です．

「確率統計I, II, III」では，確率分布やその性質，検定等の統計的推測の理論・方法などについて学びます．

----
### 例1 いんちきサイコロ

立方体の各面に 1 から 6 の数に対応する印が書かれたサイコロを投げると，1,2,3,4,5,6のそれぞれの目が出る確率は $\frac{1}{6}$ です．
でも，目によって確率の異なるいんちきサイコロがあったとしたら...



以下の `N` はサイコロを振る回数．10 から 100, 1000, 10000 などに変えてみよう．同じ N でも実行するたびに結果は変わります．

いんちきサイコロの方は，1の目が確率 $1/2$ で，他の目が $1/10$ で出ます．

In [ ]:
#@title サイコロを振る． { run: "auto" }
N = 10 #@param {type:"number"}

X = dice(N)
print()
print(f'ふつうのサイコロ N = {N}')
print(f'出目（のうち最初の方）: {X[:100]}')
print(' 出目   度数   相対度数   （真の値）')
for i in range(6):
    cnt = np.sum(X == i+1)
    print(f'   {i+1}     {cnt}     {cnt/N:.3f}     {1/6:.3f}')

P = np.ones(6)/10.0  # 1が確率 1/2, 他が 1/10
P[0] = 1.0 - np.sum(P[1:])
X = dice(N, P=P)
print()
print(f'いんちきサイコロ N = {N}: ')
print(f'出目（のうち最初の方）: {X[:100]}')
print(' 出目   度数   相対度数   （真の値）')
for i in range(6):
    cnt = np.sum(X == i+1)
    print(f'   {i+1}     {cnt}     {cnt/N:.3f}     {P[i]:.3f}')

ふつうのサイコロでも，出目には偶然の結果による偏りがある程度ありますので，上記の相対度数はちょうど $1/6$ になるわけではありません．
$N$ が小さいときは，いんちきサイコロと同じくらい偏った出目になることもあるかもしれません．

上記のようなデータが得られたときに，サイコロがふつうのまっとうなものかどうか，いんちきなものでないかどうか，を判断するには，どうしたらよいのでしょうか．




----
### 例2 ほげおのクッキー
ほげおくんはお菓子屋を営んでいます．お店の人気商品であるクッキーは，いままで直径約5cmでしたが，良からぬことを考えたほげおくんは，少し小さく作って売ることにしてしまいました．

以下の `N` の値を変えてセルを実行すると，ほげおの作るクッキーの直径のサンプルを集めて，そのヒストグラムを描くことができます．

オレンジの曲線は，平均 5.0，標準偏差 0.3 の正規分布の確率密度関数，すなわち，以下の式で $\mu = 5.0, \sigma^2 = 0.3^2$ としたものです．
$$
\frac{1}{\sqrt{2\pi\sigma^2}}\exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)
$$

ほげおくんがいんちきせずにクッキーを作ったときは，直径はこの分布にしたがっていたとすると...．集めたサンプルから，ほげおくんがいんちきしてるかどうか分かるでしょうか？

以下のセルを`N = 20` のままで何度も実行してみたり， N を大きくしてみたりして，グラフを観察しよう．

In [ ]:
#@title ほげおのクッキー屋さん． { run: "auto" }
N =  20#@param {type:"number"}
X = 0.3*np.random.randn(N) + 4.8
print('データのうち最初の10個:', end=' ')
for n in range(min(10, N)):
    print(f'{X[n]:.2f}', end=' ')

# ヒストグラム（面積1になるように描いたもの）と，正規分布の確率密度関数
fig, ax = plt.subplots(1, facecolor='white', figsize=(9, 6))
ax.hist(X, density=True, bins=np.arange(0, 10, 0.1))
xx = np.arange(0, 10, 0.01)
ax.plot(xx, Gaussian(xx, 5.0, 0.3), linewidth=3) # 平均 5 標準偏差 0.3
ax.set_xlim(3.5, 6.5)
plt.show()

print(f'N = {N}')

上記の2つの例は，いずれも，「データの裏にある確率分布（例1では6つの目が$1/6$ずつ，例2では正規分布）を考えて，出てきたデータがそれに合っているかどうか判断する」という問題となっています．
確率統計では，こういう問題への取り組み方や客観的な解答を学ぶことができます．

